In [1]:
import numpy as np
import toppra.dracula as drac
import toppra as ta
import toppra.algorithm as algo
from toppra import constraint
from toppra.dracula import RunTopp, V_MAX, A_MAX, _check_waypoints

In [2]:
import re
import ast
import numpy as np

def parse_str_arr(s):
    a = re.sub(r"([^[])\s+([^]])", r"\1, \2", s)
    return np.array(ast.literal_eval(a))

In [48]:
waypts = np.loadtxt('test_waypts_3.txt')
coeff = 0.5
vlim = coeff * np.vstack([-V_MAX, V_MAX]).T
alim = coeff * np.vstack([-A_MAX, A_MAX]).T

In [39]:
waypts

array([[-1.14793 ,  1.63924 , -0.766674, -0.793794,  0.744133,  2.9017  ,
         0.784708],
       [-1.14818 ,  1.63814 , -0.766265, -0.79508 ,  0.743997,  2.90035 ,
         0.78471 ]])

In [18]:
data = np.load('/data/toppra/20201204T144956.604020+0000.npz')
waypts = data['waypts']
vlim = data['vlim']
alim = data['alim']
# print(f"waypts:\n{waypts}\nvlim\n{vlim}\nalim\n{alim}")

In [17]:
data['vlim']

array([[-0.5383125,  0.5383125],
       [-0.5383125,  0.5383125],
       [-0.5383125,  0.5383125],
       [-0.5383125,  0.5383125],
       [-0.645975 ,  0.645975 ],
       [-0.645975 ,  0.645975 ],
       [-0.645975 ,  0.645975 ]])

In [100]:
min_pair_dist, t_sum = _check_waypoints(waypts, vlim)

In [101]:
t_sum

0.21726608429118774

In [103]:
N_samples = waypts.shape[0]
min_pair_dist, t_sum = _check_waypoints(waypts, vlim)
x = np.linspace(0, 0.15, N_samples)  # magic number
path = ta.SplineInterpolator(x, waypts.copy(), bc_type="clamped")
pc_vel = constraint.JointVelocityConstraint(vlim)
pc_acc = constraint.JointAccelerationConstraint(
    alim, discretization_scheme=constraint.DiscretizationType.Interpolation
)
instance = algo.TOPPRA(
    [pc_vel, pc_acc],
    path,
    # gridpoints=gridpoints,
    solver_wrapper="seidel",
)
jnt_traj = instance.compute_trajectory(0, 0)
jnt_traj.cspl.x

 INFO [algorithm.py : 100] No gridpoint specified. Automatically choose a gridpoint with 82 points
 INFO [algorithm.py : 185] Successfully parametrize path. Duration: 236.208, previously 0.150)
 INFO [algorithm.py : 186] Finish parametrization in 0.006 secs


array([0.00000000e+00, 3.16199586e-02, 4.60807571e-02, 6.98534902e-02,
       8.96981238e-02, 1.09219610e-01, 1.19978013e-01, 1.31548412e-01,
       1.43994573e-01, 1.57206783e-01, 1.71047139e-01, 1.85690955e-01,
       2.00384167e-01, 2.14353549e-01, 2.40340844e-01, 2.63714702e-01,
       2.86547900e-01, 2.98676786e-01, 3.11448229e-01, 3.24832863e-01,
       3.38772825e-01, 3.52667365e-01, 3.65999132e-01, 3.78726769e-01,
       3.90799480e-01, 4.02279883e-01, 4.13126165e-01, 4.32822977e-01,
       4.49881651e-01, 4.66819611e-01, 4.76227395e-01, 4.85987430e-01,
       4.95860630e-01, 5.05312706e-01, 5.14130230e-01, 5.22532177e-01,
       5.38172225e-01, 5.52096914e-01, 5.78110499e-01, 5.92391505e-01,
       6.05885490e-01, 6.17255324e-01, 6.27890502e-01, 6.38366510e-01,
       6.49605584e-01, 6.62253222e-01, 6.76260358e-01, 6.90817410e-01,
       7.04814861e-01, 7.31597208e-01, 7.45523006e-01, 7.60113422e-01,
       7.74783405e-01, 7.87926466e-01, 8.08726169e-01, 8.41283647e-01,
      

In [113]:
cs = jnt_traj.cspl
x = cs.x
vlim = vlim[:, 1]

In [114]:
vlim

array([1.0875, 1.0875, 1.0875, 1.0875, 1.305 , 1.305 , 1.305 ])

In [119]:
(np.abs(cs.derivative(1)(x))/vlim > 1).any()

False

In [2]:
waypts = np.array([
[  -1.36967,    0.70179, -0.0312555,   -1.66143, -0.0937177,    1.74777,   0.861107],
[  -1.36984,   0.719352, -0.0312674,   -1.65578, -0.0936816,    1.74725,   0.862961],
[     -1.37,    0.73693, -0.0312514,   -1.64963,  -0.093625,    1.74677,   0.866031],
[  -1.37013,   0.754249, -0.0311754,   -1.64326, -0.0936026,    1.74641,   0.871362],
[   -1.3702,   0.770595, -0.0310192,   -1.63715, -0.0936601,    1.74621,   0.879502],
[  -1.37024,   0.786312, -0.0307889,   -1.63132, -0.0937787,    1.74612,   0.890403],
[  -1.37026,   0.802591, -0.0305029,   -1.62542, -0.0938774,    1.74602,   0.904025],
[  -1.37032,   0.819782, -0.0301943,   -1.61945, -0.0938451,    1.74574,   0.919921],
[  -1.37042,   0.836643, -0.0298762,   -1.61395, -0.0936258,    1.74529,   0.935952],
[  -1.37049,   0.852481, -0.0294009,   -1.60926, -0.0933568,    1.74517,   0.950451],
[  -1.37048,   0.867246, -0.0286046,   -1.60548, -0.0931964,    1.74586,   0.962663],
[  -1.37032,   0.880987, -0.0273796,   -1.60263, -0.0932485,    1.74768,   0.972146],
[  -1.37007,   0.893714, -0.0258217,   -1.60061, -0.0934865,    1.75056,   0.978964],
[  -1.36984,   0.905399, -0.0242404,    -1.5992, -0.0937489,    1.75404,   0.983667],
[  -1.36974,   0.916055,  -0.022901,   -1.59822, -0.0938926,    1.75774,   0.986702],
[  -1.36988,   0.925686, -0.0220625,   -1.59753, -0.0937757,    1.76138,   0.988383],
[  -1.37034,   0.934221, -0.0218843,   -1.59704, -0.0933034,    1.76469,   0.988942],
[  -1.37105,    0.94166, -0.0222157,   -1.59676, -0.0925256,    1.76759,    0.98864],
[  -1.37191,   0.948023, -0.0228089,   -1.59673,  -0.091541,    1.77005,   0.987796],
[  -1.37364,   0.957582, -0.0237548,   -1.59744, -0.0894406,    1.77346,   0.985888],
[  -1.37503,   0.963847, -0.0240086,   -1.59871, -0.0876124,    1.77532,   0.984542]
])
vmax = np.array([0.538312, 0.538312, 0.538312, 0.538312, 0.645975, 0.645975, 0.645975])
amax = np.array([6.375, 3.1875,   4.25, 5.3125,  6.375,    8.5,    8.5])
vlim = np.vstack([-vmax, vmax]).T
alim = np.vstack([-amax, amax]).T

In [52]:
cs = RunTopp(waypts, vlim, alim, return_cspl=True)

 INFO [__init__.py : 60] Debug environment detected, input data saved to: /data/toppra/20201204T160314.822441+0000.npz
 INFO [algorithm.py : 100] No gridpoint specified. Automatically choose a gridpoint with 5 points
 INFO [algorithm.py : 185] Successfully parametrize path. Duration: 0.055, previously 0.150)
 INFO [algorithm.py : 186] Finish parametrization in 0.002 secs


In [22]:
np.where(np.abs(cs.derivative(1)(cs.x)) > vmax)

(array([], dtype=int64), array([], dtype=int64))

In [25]:
vmax - np.abs(cs.derivative(1)(cs.x))[15,:]

array([0.54291996, 0.01194971, 0.53422802, 0.35166617, 0.64943048,
       0.64899212, 0.19915499])

In [54]:
waypts.shape

(2, 7)

In [55]:
cs = RunTopp(waypts, vlim, alim, return_cspl=True)

 INFO [__init__.py : 60] Debug environment detected, input data saved to: /data/toppra/20201204T160332.850619+0000.npz
 INFO [algorithm.py : 100] No gridpoint specified. Automatically choose a gridpoint with 5 points
 INFO [algorithm.py : 185] Successfully parametrize path. Duration: 0.055, previously 0.150)
 INFO [algorithm.py : 186] Finish parametrization in 0.004 secs


In [56]:
new_waypts = cs(cs.x)
new_waypts.shape

(9, 7)

In [57]:
new_cs = RunTopp(new_waypts, vlim, alim, return_cspl=True)

 INFO [__init__.py : 60] Debug environment detected, input data saved to: /data/toppra/20201204T160342.125263+0000.npz
WARNING [__init__.py : 85] Duplicates found in input waypoints. This is not recommended, especially for the beginning and the end of the trajectory. Toppra might throw a controllability exception. Attempting to optimise trajectory anyway...
 INFO [algorithm.py : 100] No gridpoint specified. Automatically choose a gridpoint with 5 points
 INFO [algorithm.py : 185] Successfully parametrize path. Duration: 0.095, previously 0.150)
 INFO [algorithm.py : 186] Finish parametrization in 0.002 secs
